In [5]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.append('..')
from JKBio.epigenetics import ChIP_helper as chiphelper
from JKBio import Helper as helper
import igv
import numpy as np
import pyBigWig
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

# ChIP

In [ ]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

In [ ]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip/IRF2BP2/

In [ ]:
rename1 = {
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_1.fastq.gz",
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_2.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_1.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_2.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_1.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_2.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_1.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_2.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_1.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_2.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_1.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_2.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_1.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_2.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_1.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_2.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_1.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_2.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_1.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_2.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_1.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_2.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_1.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_2.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_1.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_2.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_1.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_2.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_1.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_2.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_1.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_2.fastq.gz"}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
! mkdir bams && gsutil -m cp "gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/*" bams

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/Chip/IRF2BP2/ && mv results results1 && sudo rm -r work

In [ ]:
# install bwa
! mkdir ref
! curl ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz -o ref/reference_droso.fna.gz
! gunzip ref/reference_droso.fna.gz
! bwa index -a bwtsw ref/reference_droso.fna

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [1.0, 0.3756814126],
[1.0, 0.2342366196],
[0.5546432407, 1.0],
[0.1601166841, 1.0],
[0.870315477, 1.0],
[1.0, 0.8742449687],
[0.7172728398, 1.0],
[1.0, 0.7713982525]

In [ ]:
# diffPeak & 
bams = !ls data/bamshuman/*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[val])

In [ ]:
scales = [1.0,
1.0,
0.5546432407,
0.1601166841,
0.870315477,
1.0,
0.7172728398,
1.0,
0.3756814126,
0.2342366196,
1.0,
1.0,
1.0,
0.8742449687,
1.0,
0.7713982525]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip/IRF2BP2/ &&\
        gsutil -m cp -r "bigwigs" gs://amlproject/Chip/IRF2BP2/

## v2

In [ ]:
#! rm -r bams && mkdir bams && gsutil -m cp "gs://transfer-amlproject/20191219_MP7659*" bams/

In [ ]:
rename = {
"20191219_MP7659_1_S1_R1_001.fastq.gz":"mp795_R1.fastq.gz",
"20191219_MP7659_1_S1_R2_001.fastq.gz":"mp795_R2.fastq.gz",
"20191219_MP7659_2_S2_R1_001.fastq.gz":"mp796_R1.fastq.gz",
"20191219_MP7659_2_S2_R2_001.fastq.gz":"mp796_R2.fastq.gz",
"20191219_MP7659_3_S3_R1_001.fastq.gz":"mp797_R1.fastq.gz",
"20191219_MP7659_3_S3_R2_001.fastq.gz":"mp797_R2.fastq.gz",
"20191219_MP7659_4_S4_R1_001.fastq.gz":"mp798_R1.fastq.gz",
"20191219_MP7659_4_S4_R2_001.fastq.gz":"mp798_R2.fastq.gz",
"20191219_MP7659_5_S5_R1_001.fastq.gz":"mp799_R1.fastq.gz",
"20191219_MP7659_5_S5_R2_001.fastq.gz":"mp799_R2.fastq.gz",
"20191219_MP7659_6_S6_R1_001.fastq.gz":"mp800_R1.fastq.gz",
"20191219_MP7659_6_S6_R2_001.fastq.gz":"mp800_R2.fastq.gz",
"20191219_MP7659_7_S7_R1_001.fastq.gz":"mp801_R1.fastq.gz",
"20191219_MP7659_7_S7_R2_001.fastq.gz":"mp801_R2.fastq.gz",
"20191219_MP7659_8_S8_R1_001.fastq.gz":"mp802_R1.fastq.gz",
"20191219_MP7659_8_S8_R2_001.fastq.gz":"mp802_R2.fastq.gz",
"20191219_MP7659_9_S9_R1_001.fastq.gz":"mp803_R1.fastq.gz",
"20191219_MP7659_9_S9_R2_001.fastq.gz":"mp803_R2.fastq.gz",
"20191219_MP7659_10_S10_R1_001.fastq.gz":"mp804_R1.fastq.gz",
"20191219_MP7659_10_S10_R2_001.fastq.gz":"mp804_R2.fastq.gz",
"20191219_MP7659_11_S11_R1_001.fastq.gz":"mp805_R1.fastq.gz",
"20191219_MP7659_11_S11_R2_001.fastq.gz":"mp805_R2.fastq.gz",
"20191219_MP7659_12_S12_R1_001.fastq.gz":"mp806_R1.fastq.gz",
"20191219_MP7659_12_S12_R2_001.fastq.gz":"mp806_R2.fastq.gz",
"20191219_MP7659_13_S13_R1_001.fastq.gz":"mp807_R1.fastq.gz",
"20191219_MP7659_13_S13_R2_001.fastq.gz":"mp807_R2.fastq.gz",
"20191219_MP7659_14_S14_R1_001.fastq.gz":"mp808_R1.fastq.gz",
"20191219_MP7659_14_S14_R2_001.fastq.gz":"mp808_R2.fastq.gz",
"20191219_MP7659_15_S15_R1_001.fastq.gz":"mp809_R1.fastq.gz",
"20191219_MP7659_15_S15_R2_001.fastq.gz":"mp809_R2.fastq.gz",
"20191219_MP7659_16_S16_R1_001.fastq.gz":"mp810_R1.fastq.gz",
"20191219_MP7659_16_S16_R2_001.fastq.gz":"mp810_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $k $v
! gsutil -m cp 'bams/*' gs://amlproject/Chip_AML/IRF2BP2/V2/    

In [ ]:
! gsutil ls gs://amlproject/Chip_AML/IRF2BP2/

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/Chip/IRF2BP2/ && sudo rm -r work && mv results results2

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [1.0,0.7976494266],
[0.0060104845,1.0],
[0.6415816327,1.0],
[0.6370573492,1.0],
[1.0,0.8327053276],
[1.0,0.2711054926],
[0.8111195721,1.0],
[1.0,0.8338050685],

In [ ]:
# diffPeak & 
bams = !ls data/bamshuman/*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[val])

In [ ]:
scales = [1.0,
0.006010484524,
0.6415816327,
0.6370573492,
1.0,
1.0,
0.8111195721,
1.0,
0.7976494266,
1.0,
1.0,
1.0,
0.8327053276,
0.2711054926,
1.0,
0.8338050685]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip/IRF2BP2/ &&\
        gsutil -m cp "bigwigs/mp*" gs://amlproject/Chip/IRF2BP2/bigwigs

### V3

In [ ]:
bamfolder="histone_bams/"

In [ ]:
! mkdir histone_bams && gsutil cp "gs://transfer-amlproject/*.fastq.gz" $bamfolder

In [ ]:
rename = {
    "20200103_1_MP7693_S6_R1_001.fastq.gz":"mp811_R1.fastq.gz",
    "20200103_1_MP7693_S6_R2_001.fastq.gz":"mp811_R2.fastq.gz",
    "20200103_2_MP7693_S7_R1_001.fastq.gz":"mp812_R1.fastq.gz",
    "20200103_2_MP7693_S7_R2_001.fastq.gz":"mp812_R2.fastq.gz",
    "20200103_3_MP7693_S8_R1_001.fastq.gz":"mp813_R1.fastq.gz",
    "20200103_3_MP7693_S8_R2_001.fastq.gz":"mp813_R2.fastq.gz",
    "20200103_4_MP7693_S9_R1_001.fastq.gz":"mp814_R1.fastq.gz",
    "20200103_4_MP7693_S9_R2_001.fastq.gz":"mp814_R2.fastq.gz",
    "20200103_5_MP7693_S10_R1_001.fastq.gz":"mp815_R1.fastq.gz",
    "20200103_5_MP7693_S10_R2_001.fastq.gz":"mp815_R2.fastq.gz",
    "20200103_6_MP7693_S11_R1_001.fastq.gz":"mp816_R1.fastq.gz",
    "20200103_6_MP7693_S11_R2_001.fastq.gz":"mp816_R2.fastq.gz",
    "20200103_7_MP7693_S12_R1_001.fastq.gz":"mp817_R1.fastq.gz",
    "20200103_7_MP7693_S12_R2_001.fastq.gz":"mp817_R2.fastq.gz",
    "20200103_8_MP7693_S13_R1_001.fastq.gz":"mp818_R1.fastq.gz",
    "20200103_8_MP7693_S13_R2_001.fastq.gz":"mp818_R2.fastq.gz",
    "20200103_9_MP7693_S14_R1_001.fastq.gz":"mp819_R1.fastq.gz",
    "20200103_9_MP7693_S14_R2_001.fastq.gz":"mp819_R2.fastq.gz",
    "20200103_10_MP7693_S15_R1_001.fastq.gz":"mp820_R1.fastq.gz",
    "20200103_10_MP7693_S15_R2_001.fastq.gz":"mp820_R2.fastq.gz",
    "20200103_11_MP7693_S16_R1_001.fastq.gz":"mp821_R1.fastq.gz",
    "20200103_11_MP7693_S16_R2_001.fastq.gz":"mp821_R2.fastq.gz",
    "20200103_12_MP7693_S17_R1_001.fastq.gz":"mp822_R1.fastq.gz",
    "20200103_12_MP7693_S17_R2_001.fastq.gz":"mp822_R2.fastq.gz",
    "20200103_13_MP7693_S18_R1_001.fastq.gz":"mp823_R1.fastq.gz",
    "20200103_13_MP7693_S18_R2_001.fastq.gz":"mp823_R2.fastq.gz",
    "20200103_14_MP7693_S19_R1_001.fastq.gz":"mp824_R1.fastq.gz",
    "20200103_14_MP7693_S19_R2_001.fastq.gz":"mp824_R2.fastq.gz",
    "20200103_15_MP7693_S20_R1_001.fastq.gz":"mp825_R1.fastq.gz",
    "20200103_15_MP7693_S20_R2_001.fastq.gz":"mp825_R2.fastq.gz",
    "20200103_16_MP7693_S21_R1_001.fastq.gz":"mp826_R1.fastq.gz",
    "20200103_16_MP7693_S21_R2_001.fastq.gz":"mp826_R2.fastq.gz",
    "20200103_17_MP7693_S22_R1_001.fastq.gz":"mp827_R1.fastq.gz",
    "20200103_17_MP7693_S22_R2_001.fastq.gz":"mp827_R2.fastq.gz",
    "20200103_18_MP7693_S23_R1_001.fastq.gz":"mp828_R1.fastq.gz",
    "20200103_18_MP7693_S23_R2_001.fastq.gz":"mp828_R2.fastq.gz",
    "20200103_19_MP7693_S24_R1_001.fastq.gz":"mp829_R1.fastq.gz",
    "20200103_19_MP7693_S24_R2_001.fastq.gz":"mp829_R2.fastq.gz",
    "20200103_20_MP7693_S25_R1_001.fastq.gz":"mp830_R1.fastq.gz",
    "20200103_20_MP7693_S25_R2_001.fastq.gz":"mp830_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $bamfolder$k $bamfolder$v

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq $bamfolder

In [ ]:
a = ! ls $bamfolder

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
 

In [ ]:
! gsutil -m cp -r results gs:// 
! gsutil -m cp $bamfolder'/*' gs://amlproject/Chip_AML/IRF2BP2/V3/

## RNA

In [ ]:
a = !gsutil ls gs://amlproject/slamseq/IRF2BP2/

In [ ]:
rename1 = {
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_2.fastq.gz'}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
a

In [ ]:
import TerraFunction as terra

In [ ]:
terra.list_blobs_with_prefix("amlproject","slamseq/IRF2BP2/")

In [ ]:
terra.uploadFromFolder("amlproject", "slamseq/IRF2BP2/", workspace="broad-firecloud-ccle/hg38_RNAseq", 
                       fformat="fastq12", source="maximP",samplesetname="slamseq_IRF2BP2")

In [ ]:
import sys
sys.path.insert(0, '../ccle_processing/')
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
from Helper import *
import dalmatian as dm

In [ ]:
samplesetname = 'slamseq_IRF2BP2'
refworkspace="broad-firecloud-ccle/hg38_RNAseq"
refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
submission_id = refwm.create_submission("slamdunk", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id1)